# 🦙 LLaMA 3 8B - Validation Équipe\n**Notebook de test et validation sur Google Colab**\n\n👥 **Équipe de développement** - Synchronisation avec AWS

## 🔧 1. Configuration de l'Environnement

In [ ]:
# Installation des dépendances\n!pip install -q torch>=2.1.0 transformers>=4.35.0 accelerate>=0.24.0 bitsandbytes>=0.41.0 huggingface-hub>=0.19.0 gradio>=4.0.0

In [ ]:
# Montage Google Drive pour la persistance\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\nprint(\"✅ Google Drive monté\")

## 🔄 2. Synchronisation du Code depuis AWS/GitHub

In [ ]:
# CHOISISSEZ UNE OPTION :\n\nprint(\"Options de synchronisation :\")\nprint(\"1. Depuis GitHub (recommandé)\")\nprint(\"2. Depuis Google Drive (si vous avez sync manuel)\")\nprint(\"3. Téléchargement direct\")\n\noption = 1  # ← Changez selon votre préférence

In [ ]:
if option == 1:\n    # Synchronisation depuis GitHub\n    !rm -rf llama-project\n    !git clone https://github.com/VOTRE_USERNAME/llama-3-8b-team.git llama-project\n    %cd llama-project\n    print(\"✅ Code synchronisé depuis GitHub\")

## 🖥️ 3. Vérification du Matériel

In [ ]:
import torch\nimport psutil\n\nprint(\"🖥️ INFORMATIONS MATÉRIEL COLAB\")\nprint(\"=\" * 50)\n\n# CPU\nprint(f\"✅ CPU: {torch.get_num_threads()} cores\")\nprint(f\"✅ RAM: {psutil.virtual_memory().total / 1e9:.1f} GB\")\n\n# GPU\nif torch.cuda.is_available():\n    gpu_name = torch.cuda.get_device_name(0)\n    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9\n    print(f\"✅ GPU: {gpu_name}\")\n    print(f\"✅ VRAM: {gpu_memory:.1f} GB\")\nelse:\n    print(\"❌ Aucun GPU détecté - Vérifiez la configuration Colab\")\n\nprint(\"\\n🎯 Convient pour LLaMA 3 8B (4-bit): OUI\" if torch.cuda.is_available() and gpu_memory >= 12 else \"\\n⚠️  GPU limité pour LLaMA 3 8B\")

## 🧪 4. Test du Modèle LLaMA 3 8B

In [ ]:
# Importation de notre code\nimport sys\nsys.path.append('/content/llama-project/src')\n\nfrom model_loader import ModelLoader\n\nprint(\"✅ Code de l'équipe importé\")

In [ ]:
# Chargement du modèle avec 4-bit quantization\nprint(\"🔄 Chargement de LLaMA 3 8B...\")\n\nloader = ModelLoader()\nmodel, tokenizer = loader.load_model(quantize_4bit=True)\n\nprint(\"✅ Modèle chargé avec succès!\")\nprint(f\"📊 Modèle sur: {next(model.parameters()).device}\")

## 📊 5. Tests de Performance

In [ ]:
import time\n\ndef benchmark_generation(prompts, num_runs=2):\n    results = []\n    \n    for i, prompt in enumerate(prompts):\n        print(f\"\\n🧪 Test {i+1}/{len(prompts)}: '{prompt[:50]}...'\")\n        \n        times = []\n        for run in range(num_runs):\n            start_time = time.time()\n            \n            response = loader.generate_text(\n                prompt,\n                max_length=200,\n                temperature=0.7\n            )\n            \n            generation_time = time.time() - start_time\n            times.append(generation_time)\n            \n            if run == 0:  # Afficher seulement la première réponse\n                print(f\"   Réponse: {response}\")\n        \n        avg_time = sum(times) / len(times)\n        results.append({\n            'prompt': prompt,\n            'avg_time': avg_time,\n            'tokens_per_second': len(response.split()) / avg_time\n        })\n        \n        print(f\"   ⏱️  Temps moyen: {avg_time:.2f}s\")\n        print(f\"   🚀 Tokens/seconde: {len(response.split()) / avg_time:.1f}\")\n    \n    return results\n\n# Tests de performance\ntest_prompts = [\n    \"Explique-moi l'apprentissage automatique en une phrase:\",\n    \"Qu'est-ce que la transformée de Fourier? Réponse courte:\",\n    \"Donne-moi 3 conseils pour améliorer mon code Python:\"\n]\n\nprint(\"📊 LANCEMENT DES TESTS DE PERFORMANCE\")\nprint(\"=\" * 50)\n\nperformance_results = benchmark_generation(test_prompts, num_runs=2)

In [ ]:
# Résumé des performances\nprint(\"\\n📈 RÉSUMÉ DES PERFORMANCES\")\nprint(\"=\" * 40)\n\navg_generation_time = sum(r['avg_time'] for r in performance_results) / len(performance_results)\navg_tokens_per_second = sum(r['tokens_per_second'] for r in performance_results) / len(performance_results)\n\nprint(f\"⏱️  Temps de génération moyen: {avg_generation_time:.2f}s\")\nprint(f\"🚀 Tokens/seconde moyen: {avg_tokens_per_second:.1f}\")\nprint(f\"💾 Mémoire GPU utilisée: {torch.cuda.memory_allocated() / 1e9:.1f} GB\")\n\n# Évaluation\nif avg_tokens_per_second > 5:\n    print(\"\\n🎉 PERFORMANCES: EXCELLENTES\")\nelif avg_tokens_per_second > 2:\n    print(\"\\n✅ PERFORMANCES: BONNES\")\nelse:\n    print(\"\\n⚠️  PERFORMANCES: LIMITÉES\")

## 💾 6. Sauvegarde des Résultats

In [ ]:
# Sauvegarde des résultats\nimport json\nfrom datetime import datetime\n\n# Création du rapport\nvalidation_report = {\n    \"date\": datetime.now().isoformat(),\n    \"environment\": {\n        \"gpu\": torch.cuda.get_device_name(0) if torch.cuda.is_available() else \"None\",\n        \"gpu_memory_gb\": torch.cuda.get_device_properties(0).total_memory / 1e9 if torch.cuda.is_available() else 0,\n        \"colab_runtime\": \"GPU\" if torch.cuda.is_available() else \"CPU\"\n    },\n    \"performance\": performance_results,\n    \"summary\": {\n        \"avg_generation_time\": avg_generation_time,\n        \"avg_tokens_per_second\": avg_tokens_per_second,\n        \"gpu_memory_used_gb\": torch.cuda.memory_allocated() / 1e9 if torch.cuda.is_available() else 0\n    },\n    \"validation_status\": \"SUCCESS\" if avg_tokens_per_second > 2 else \"WARNING\"\n}\n\n# Sauvegarde sur Google Drive\nreport_path = \"/content/drive/MyDrive/llama_team_validation.json\"\nwith open(report_path, 'w') as f:\n    json.dump(validation_report, f, indent=2)\n\nprint(f\"✅ Rapport sauvegardé: {report_path}\")\nprint(f\"📊 Statut validation: {validation_report['validation_status']}\")\n\n# Affichage du rapport\nprint(\"\\n\" + \"=\"*50)\nprint(\"🎉 VALIDATION TERMINÉE AVEC SUCCÈS!\")\nprint(\"=\"*50)\nprint(f\"📅 Date: {validation_report['date'][:19]}\")\nprint(f\"🖥️  Environment: {validation_report['environment']['colab_runtime']}\")\nprint(f\"⏱️  Performance: {validation_report['summary']['avg_tokens_per_second']:.1f} tokens/sec\")\nprint(f\"💾 Mémoire: {validation_report['summary']['gpu_memory_used_gb']:.1f} GB\")\nprint(\"\\n🚀 Prêt pour le développement en équipe!\")